## This notebook implements two-way grid square to lat/lon conversion.

Based on description at https://en.wikipedia.org/wiki/Maidenhead_Locator_System

In [1]:
import string
import numpy as np
import pandas as pd

In [2]:
# Create string lookup lists for each of the codes.
alpha_upper = np.char.array([x for x in string.ascii_uppercase])
alpha_lower = np.char.array([x for x in string.ascii_lowercase])
nr_str      = np.char.array(['{!s}'.format(x) for x in range(10)])

def latlon2gridsquare(lat,lon,precision=6):
    """
    Calculates gridsquares from lat,lon pairs.
    This routine is vectorized.
    
    Precision indicates the desired number of Grid Square characters,
    and must be an even number. 4 is often used in HF communications,
    and 6 is standard for VHF/UHF. Any two locations within the same 
    6-character grid square are no more than 12 km apart.
    """
    #### Pre-process
    lats = np.array(lat)
    lons = np.array(lon)
    
    # Make sure all input lons are between -180 and +180 deg.
    tf = lons > 180.
    lons[tf] -= 360.

    # Define zLats that start at 0 at the south pole
    # Define zLons that start at 0 at the antimeridian of Greenwich
    zLats = lats +  90.
    zLons = lons + 180.


    # Calculate the field (a.k.a. first 2 letters)
    base = 18.
    container_size_lat = 180.
    container_size_lon = 360.
    subdivide_size_lat = container_size_lat / base
    subdivide_size_lon = container_size_lon / base

    zLats_rem        = zLats % container_size_lat
    zLons_rem        = zLons % container_size_lon

    lat_code_inx     = np.array(np.floor(zLats_rem / subdivide_size_lat),dtype=np.int)
    lon_code_inx     = np.array(np.floor(zLons_rem / subdivide_size_lon),dtype=np.int)

    lon_code  = alpha_upper[lon_code_inx]
    lat_code  = alpha_upper[lat_code_inx]
    this_code = lon_code + lat_code

    grid_square = this_code
    curr_prec   = 2

    # Square, subsquare, extended square, and beyond...
    while curr_prec < precision:
        # Determine if we use base 10 numerics for base 24 alpha
        # for this portion of the code.
        alpha        = not bool((curr_prec/2) % 2)
        if alpha:
            base     = 24
            str_code = alpha_lower
        else:
            base     = 10.
            str_code = nr_str

        container_size_lat     = subdivide_size_lat
        container_size_lon     = subdivide_size_lon
        subdivide_size_lat     = container_size_lat / base
        subdivide_size_lon     = container_size_lon / base

        zLats_rem        = zLats_rem % container_size_lat
        zLons_rem        = zLons_rem % container_size_lon

        lat_code_inx     = np.array(np.floor(zLats_rem / subdivide_size_lat),dtype=np.int)
        lon_code_inx     = np.array(np.floor(zLons_rem / subdivide_size_lon),dtype=np.int)

        lon_code    = str_code[lon_code_inx]
        lat_code    = str_code[lat_code_inx]
        this_code   = lon_code + lat_code
        grid_square = grid_square + this_code
        curr_prec  += 2
        
    return grid_square

In [3]:
# Some sample grid square - lat/lon pairs.
test_dct          = {}

tmp               = {}
test_dct['aa7bq'] = tmp
tmp['lat']        =   33.698256
tmp['lon']        = -111.891495
tmp['gs']         = 'DM43bq'

tmp               = {}
test_dct['w2naf'] = tmp
tmp['lat']        =  40.811667
tmp['lon']        = -74.210000
tmp['gs']         = 'FN20vt'

tmp               = {}
test_dct['k2mff'] = tmp
tmp['lat']        =  40.7429
tmp['lon']        = -74.1770
tmp['gs']         = 'FN20vr'

# Put everything into numpy arrays to test multiple conversions.
lats, lons, gss = [], [], []
for key,item in test_dct.iteritems():
    lats.append(item['lat'])
    lons.append(item['lon'])
    gss.append(item['gs'])

In [4]:
# Test a single location - W2NAF (40.811667, -74.210000, FN20vt)
grid_square = latlon2gridsquare(40.811667, -74.210000)
print 'Grid Square: ', grid_square

# Test multiple locations
grid_square = latlon2gridsquare(lats,lons)
print 'Grid Square: ', grid_square

Grid Square:  FN20vt
Grid Square:  ['FN20vt' 'FN20vr' 'DM43bq']


In [5]:
gss = np.char.array(gss).lower()

# Create string lookup lists for each of the codes.
alpha_pd = pd.Series(range(26),index=alpha_lower)

# Make sure precision of array is uniform.
precs = np.array([len(x) for x in gss])
precision = precs.max()
if np.unique(precs).size != 1:
    # If a vector of gridsqaures is given without uniform precision,
    # there should be a subroutine to pad the gridsquares with less
    # to the center of the cell. e.g. FN20 --> FN20mm
    raise()
    
pos   = 0
zLat  = 0.
zLon  = 0.

base = 18.
container_size_lat = 180.
container_size_lon = 360.

while pos < precision:
    
    tmp_0 = gss.ljust(pos+2)
    tmp_1 = gss.ljust(pos)
    if pos == 0: tmp_1[:] = ''
    code  = tmp_0.lstrip(tmp_1)
    
    lon_code = code.ljust(1)
    lat_code = code.lstrip(lon_code)
    
    alpha        = not bool(pos/2 % 2)
    
    if alpha:
        lon_inx = np.array(alpha_pd.loc[lon_code].tolist(),dtype=np.float)
        lat_inx = np.array(alpha_pd.loc[lat_code].tolist(),dtype=np.float)
        if pos != 0: base = 24.
    else:
        lon_inx = np.array(lon_code,dtype=np.float)
        lat_inx = np.array(lat_code,dtype=np.float)
        base = 10.
    
    subdivide_size_lat = container_size_lat / base
    subdivide_size_lon = container_size_lon / base
    
    zLat += (subdivide_size_lat * lat_inx)
    zLon += (subdivide_size_lon * lon_inx)
    print 'Pos: {!s}, dLat: {!s}, dLon: {!s}'.format(pos,subdivide_size_lat,subdivide_size_lon)
 
    container_size_lat = subdivide_size_lat
    container_size_lon = subdivide_size_lon
     
    pos += 2

lat = zLat -  90.
lon = zLon - 180.

print lat
print lon

Pos: 0, dLat: 10.0, dLon: 20.0
Pos: 2, dLat: 1.0, dLon: 2.0
Pos: 4, dLat: 0.0416666666667, dLon: 0.0833333333333
[ 40.79166667  40.70833333  33.66666667]
[ -74.25        -74.25       -111.91666667]


In [6]:
test_dct

{'aa7bq': {'gs': 'DM43bq', 'lat': 33.698256, 'lon': -111.891495},
 'k2mff': {'gs': 'FN20vr', 'lat': 40.7429, 'lon': -74.177},
 'w2naf': {'gs': 'FN20vt', 'lat': 40.811667, 'lon': -74.21}}